<a href="https://colab.research.google.com/github/sreejan226/Assign1/blob/main/2.8/code_example_baseline_v2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
monitor='val_accuracy'
epochs=30
batch_size=16
input_shape=(128, 128, 3) # please resize it to (224,224,3) if you have enough RAM
Verbose=True

This source code requires a **HIGH RAM** machine.

You might need to install this on your system:

apt-get install python3-opencv git

In [2]:
import os

if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1838, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 1838 (delta 158), reused 171 (delta 88), pack-reused 1588 (from 1)
Receiving objects: 100% (1838/1838), 15.70 MiB | 22.77 MiB/s, done.
Resolving deltas: 100% (1271/1271), done.
Processing /content/k
  Preparing metadata (setup.py) ... done
  Created wheel for cai: filename=cai-0.1.7-py3-none-any.whl size=61387 sha256=85fe051a7588bcf71c5d143dd45332fa2f6755ab2bcb60ff31f055e19724f920
  Stored in directory: /tmp/pip-ephem-wheel-cache-msquypj9/wheels/44/d3/04/5889b6c90ac4bd1e037376d19946cca1579251cf3209e6aed0
Successfully built cai


In [3]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

import skimage
print('skimage version',  skimage.__version__)
import tensorflow as tf
print(tf.__version__)
import sys
import cai
import cai.datasets
import cai.densenet
import cai.util
import cai.models

Python version
3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Version info.
sys.version_info(major=3, minor=12, micro=11, releaselevel='final', serial=0)
skimage version 0.25.2
2.19.0


In [4]:
url_zip_file="https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file,
    expected_folder_name=expected_folder_name, Verbose=Verbose)

Downloading:  https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded  to  plant_leaf.zip
Decompressing into:  plant_leaf


In [5]:
import tensorflow
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [6]:
!rm -r plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves -R
data_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"
print(os.listdir(data_dir))

['Strawberry___healthy', 'Potato___Early_blight', 'Tomato___Leaf_Mold', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Cherry___Powdery_mildew', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___healthy', 'Potato___healthy', 'Potato___Late_blight', 'Pepper,_bell___healthy', 'Peach___Bacterial_spot', 'Raspberry___healthy', 'Tomato___Septoria_leaf_spot', 'Tomato___Bacterial_spot', 'Blueberry___healthy', 'Apple___Cedar_apple_rust', 'Tomato___Early_blight', 'Tomato___healthy', 'Grape___Esca_(Black_Measles)', 'Orange___Haunglongbing_(Citrus_greening)', 'Tomato___Target_Spot', 'Apple___Black_rot', 'Tomato___Tomato_mosaic_virus', 'Apple___Apple_scab', 'Soybean___healthy', 'Cherry___healthy', 'Corn___Northern_Leaf_Blight', 'Tomato___Late_blight', 'Grape___Black_rot', 'Apple___healthy', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Strawberry___Leaf_scorch', 'Corn___Common_ru

In [7]:
def compiled_two_path_inception_v3(
  classes=1000,
  input_shape=input_shape,
  two_paths_first_block=False,
  two_paths_second_block=False,
  max_mix_idx=10):
  base_model = cai.models.two_path_inception_v3(
    include_top=False,
    weights=None,
    input_shape=input_shape,
    pooling=None,
    two_paths_first_block=two_paths_first_block,
    two_paths_second_block=two_paths_second_block,
    max_mix_idx=max_mix_idx)
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(38, name='preprediction')(x)
  predictions = Activation('softmax',name='prediction')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  model.compile(loss='categorical_crossentropy',
  optimizer = 'sgd',
  metrics=['accuracy'])
  return model

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7, root_dir=data_dir, lab=False,
  verbose=Verbose, bipolar=False, base_model_name='plant_leaf',
  training_size=0.6, validation_size=0.2, test_size=0.2,
  target_size=(input_shape[0],input_shape[1]),
  has_training=True, has_validation=True, has_testing=True,
  smart_resize=True)

Loading  38  classes.
smart resize is enabled.
loading train images


In [ ]:
print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

In [ ]:
for max_mix_idx in [5]: # range(-1,10,1):
    basefilename = 'Schuler-baseline-v2.8-'+str(max_mix_idx)
    print('Running: '+basefilename)
    model = compiled_two_path_inception_v3(
      classes=38,
      input_shape=input_shape,
      two_paths_first_block=False,
      two_paths_second_block=False,
      max_mix_idx=max_mix_idx)
    best_result_file_name = basefilename+'-best-result.hdf5'
    save_best = tensorflow.keras.callbacks.ModelCheckpoint(
      filepath=best_result_file_name,
      monitor=monitor,
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='max',
      save_freq='epoch')
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,
      validation_data=(val_x,val_y),
      callbacks=[save_best],
      class_weight = classweight
    )
    print('Testing Last Model: '+basefilename)
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Best Model Results: '+basefilename)
    model = tensorflow.keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Finished: '+basefilename)